# ETL_EDA    
This file records the process of acquiring raw data, traforming them, and loading them into a MongoDB. The data are store (almost) in their raw form.

## 1. Raw Data from LAPD     
The dataset is a continuously updated file in csv format. It contains City of Los Angeles arrest data from 2010 to present. It can be retrieved simply by making requests without parameters. For more complicated API call, you may need to add query parameters.

In [1]:
import logging
import pandas as pd
from sodapy import Socrata
import utils

# download data
CRIME_SOURCE = "data.lacity.org"
client0 = Socrata(CRIME_SOURCE, None)
results = client0.get("yru6-6re4",where="arst_date > '2019-11-30T00:00:00.000'") # wait to be confirmed
print(results[:2])
print('...')
print(results[-2:])

[{'rpt_id': '5811793', 'arst_date': '2019-12-01T00:00:00.000', 'time': '0311', 'area': '15', 'area_desc': 'N Hollywood', 'rd': '1565', 'age': '26', 'sex_cd': 'M', 'descent_cd': 'H', 'chrg_grp_cd': '22', 'grp_description': 'Driving Under Influence', 'arst_typ_cd': 'M', 'charge': '23152(A)VC', 'chrg_desc': 'DRUNK DRIVING ALCOHOL/DRUGS', 'location': 'VINELAND                     AV', 'crsst': 'HOLLYWOOD                    FY', 'location_1': {'latitude': '34.1468', 'longitude': '-118.3703', 'human_address': '{"address": "", "city": "", "state": "", "zip": ""}'}}, {'rpt_id': '5811921', 'arst_date': '2019-12-01T00:00:00.000', 'time': '0300', 'area': '13', 'area_desc': 'Newton', 'rd': '1307', 'age': '21', 'sex_cd': 'M', 'descent_cd': 'B', 'chrg_grp_cd': '04', 'grp_description': 'Aggravated Assault', 'arst_typ_cd': 'F', 'charge': '245(D)(1)PC', 'chrg_desc': 'ASSAULT W/FIREARM ON PO OR FIREMAN', 'location': '700    CROCKER                      ST', 'location_1': {'latitude': '34.0401', 'longitu

## 2. Raw Data to Documents/Dicts      
Datetime conversion is made and age column is changed to numeric data type. Now the data can be easily converted to a list of dicts which is what we want for the MongoDB.

In [2]:
from datetime import datetime

df = pd.DataFrame.from_records(results)
df['arst_date'] = pd.to_datetime(df['arst_date'])
df['hour'] = df['time'].apply(lambda x: datetime.strptime(x, '%H%M').time().hour)
df['age'] = pd.to_numeric(df['age'])

df.head()

,rpt_id,arst_date,time,area,area_desc,rd,age,sex_cd,descent_cd,chrg_grp_cd,grp_description,arst_typ_cd,charge,chrg_desc,location,crsst,location_1,hour
0,5811793,2019-12-01,0311,15,N Hollywood,1565,26,M,H,22,Driving Under Influence,M,23152(A)VC,DRUNK DRIVING ALCOHOL/DRUGS,VINELAND AV,HOLLYWOOD FY,"{'latitude': '34.1468', 'longitude': '-118.370...",3
1,5811921,2019-12-01,0300,13,Newton,1307,21,M,B,04,Aggravated Assault,F,245(D)(1)PC,ASSAULT W/FIREARM ON PO OR FIREMAN,700 CROCKER ST,NaN,"{'latitude': '34.0401', 'longitude': '-118.245...",3
2,5812227,2019-12-01,1540,13,Newton,1362,28,M,B,05,Burglary,F,459PC,BURGLARY,400 E 49TH ST,NaN,"{'latitude': '33.9988', 'longitude': '-118.269...",15
3,190920652,2019-12-01,0225,09,Van Nuys,0914,45,M,H,13,Prostitution/Allied,M,647(B)(2)PC,ENGAGE/SOLICIT SEX FOR MONEY/ETC,SEPULVEDA BL,HART ST,"{'latitude': '34.1976', 'longitude': '-118.466...",2
4,5812327,2019-12-01,2015,06,Hollywood,0646,39,M,B,03,Robbery,F,211PC,ROBBERY,COSMO,HOLLYWOOD BL,"{'latitude': '34.1016', 'longitude': '-118.328...",20


## 3. Upsert MongoDB         
To avoid duplicate data entries between each run, the de-duplication mechanism makes sure that only a new record with a different report_id (rpt_id) will be inserted to the database. 

In [3]:
import pymongo
client = pymongo.MongoClient()
db = client.get_database("crime")
collection = db.get_collection("crime")
update_count = 0
for record in df.to_dict('records'):
    result = collection.replace_one(
        filter = {'rpt_id': record['rpt_id']},    # locate the document if exists
        replacement = record,                         # latest document
        upsert=True)
    if result.matched_count > 0:
        update_count += 1
print(f"rows={df.shape[0]}, update={update_count}, "
      f"insert={df.shape[0]-update_count}")

rows=990, update=990, insert=0
